# Обучение на RL за балансиране на Cartpole

Този ноутбук е част от [Учебната програма за начинаещи в ИИ](http://aka.ms/ai-beginners). Той е вдъхновен от [официалния урок на PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) и [тази имплементация на Cartpole с PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

В този пример ще използваме RL, за да обучим модел да балансира прът върху количка, която може да се движи наляво и надясно по хоризонтална ос. Ще използваме средата [OpenAI Gym](https://www.gymlibrary.ml/), за да симулираме пръта.

> **Забележка**: Можете да изпълните кода от този урок локално (например чрез Visual Studio Code), като в този случай симулацията ще се отвори в нов прозорец. Ако изпълнявате кода онлайн, може да се наложи да направите някои промени в кода, както е описано [тук](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Ще започнем, като се уверим, че Gym е инсталиран:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Сега нека създадем средата CartPole и да видим как да работим с нея. Една среда има следните свойства:

* **Action space** е наборът от възможни действия, които можем да изпълним на всяка стъпка от симулацията
* **Observation space** е пространството на наблюденията, които можем да направим


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Нека видим как работи симулацията. Следният цикъл изпълнява симулацията, докато `env.step` не върне флага за прекратяване `done`. Ще избираме действия на случаен принцип, използвайки `env.action_space.sample()`, което означава, че експериментът вероятно ще се провали много бързо (средата CartPole приключва, когато скоростта на CartPole, неговата позиция или ъгъл излязат извън определени граници).

> Симулацията ще се отвори в нов прозорец. Можете да изпълните кода няколко пъти и да наблюдавате как се държи.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Можете да забележите, че наблюденията съдържат 4 числа. Те са:
- Позиция на количката
- Скорост на количката
- Ъгъл на пръта
- Скорост на въртене на пръта

`rew` е наградата, която получаваме на всяка стъпка. В средата CartPole можете да видите, че получавате 1 точка за всяка стъпка от симулацията, а целта е да максимизирате общата награда, т.е. времето, през което CartPole успява да се балансира, без да падне.

По време на обучението чрез подсилване, нашата цел е да обучим **политика** $\pi$, която за всяко състояние $s$ ще ни казва какво действие $a$ да предприемем, т.е. $a = \pi(s)$.

Ако искате вероятностно решение, можете да мислите за политиката като връщаща набор от вероятности за всяко действие, т.е. $\pi(a|s)$ би означавало вероятността да предприемем действие $a$ в състояние $s$.

## Метод на градиента на политиката

В най-простия алгоритъм за обучение чрез подсилване, наречен **Градиент на политиката**, ще обучим невронна мрежа да предсказва следващото действие.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Ще обучим мрежата, като проведем много експерименти и актуализираме мрежата след всяко изпълнение. Нека дефинираме функция, която ще проведе експеримента и ще върне резултатите (т.нар. **следа**) - всички състояния, действия (и препоръчаните им вероятности) и награди:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Можете да изпълните един епизод с нетренирана мрежа и да наблюдавате, че общата награда (или дължината на епизода) е много ниска:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Един от трудните аспекти на алгоритъма за градиент на политика е използването на **дисконтни награди**. Идеята е, че изчисляваме вектора на общите награди на всяка стъпка от играта и по време на този процес дисконтраме ранните награди, използвайки някакъв коефициент $gamma$. Също така нормализираме получения вектор, защото ще го използваме като тегло, за да повлияем на нашето обучение:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Сега да започнем истинското обучение! Ще проведем 300 епизода, като при всеки епизод ще изпълним следното:

1. Провеждаме експеримента и събираме следата.
1. Изчисляваме разликата (`gradients`) между предприетите действия и предсказаните вероятности. Колкото по-малка е разликата, толкова по-уверени сме, че сме избрали правилното действие.
1. Изчисляваме дисконтирани награди и умножаваме градиентите по дисконтираните награди - това гарантира, че стъпките с по-високи награди ще имат по-голямо влияние върху крайния резултат в сравнение с тези с по-ниски награди.
1. Очакваните целеви действия за нашата невронна мрежа ще бъдат частично взети от предсказаните вероятности по време на изпълнението и частично от изчислените градиенти. Ще използваме параметъра `alpha`, за да определим до каква степен градиентите и наградите се вземат предвид - това се нарича *скорост на учене* на алгоритъма за подсилване.
1. Накрая обучаваме мрежата върху състоянията и очакваните действия и повтаряме процеса.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Сега нека стартираме епизода с рендериране, за да видим резултата:


In [ ]:
_ = run_episode(render=True)

Надявам се, че вече виждате, че пръчката може да се балансира доста добре!

## Модел Actor-Critic

Моделът Actor-Critic е по-нататъшно развитие на градиентите на политиката, при което изграждаме невронна мрежа, за да научим както политиката, така и оценените награди. Мрежата ще има два изхода (или можете да я разглеждате като две отделни мрежи):
* **Actor** ще препоръчва действието, което да се предприеме, като ни предоставя разпределението на вероятностите за състоянието, както в модела с градиенти на политиката.
* **Critic** ще оценява каква би била наградата от тези действия. Той връща общите оценени награди в бъдеще за даденото състояние.

Нека дефинираме такъв модел:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Ще трябва леко да модифицираме функциите `discounted_rewards` и `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Сега ще изпълним основния цикъл за обучение. Ще използваме ръчен процес на обучение на мрежата чрез изчисляване на подходящи функции за загуба и актуализиране на параметрите на мрежата:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Основни моменти

В тази демонстрация разгледахме два алгоритъма за подсилено обучение: прост градиент на политика и по-сложния актьор-критик. Можете да видите, че тези алгоритми работят с абстрактни понятия като състояние, действие и награда - което ги прави приложими в много различни среди.

Подсиленото обучение ни позволява да научим най-добрата стратегия за решаване на даден проблем, като се основаваме единствено на крайната награда. Фактът, че не се нуждаем от етикетирани набори от данни, ни позволява да повтаряме симулациите многократно, за да оптимизираме нашите модели. Въпреки това, все още има много предизвикателства в подсиленото обучение, които можете да изучите, ако решите да се задълбочите в тази интересна област на изкуствения интелект.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
